In [ ]:
import re
import dns.resolver
import smtplib
import pandas as pd

In [ ]:
def is_valid_email_format(email):
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return re.match(regex, email) is not None
    
def get_dns_records(domain):
    records = {}
    
    # Get MX records
    try:
        mx_records = dns.resolver.resolve(domain, 'MX')
        records['MX'] = [mx.exchange.to_text() for mx in mx_records]
    except Exception as e:
        records['MX'] = f"Error: {e}"
    
    # Get A records
    try:
        a_records = dns.resolver.resolve(domain, 'A')
        records['A'] = [a.address for a in a_records]
    except Exception as e:
        records['A'] = f"Error: {e}"
    
    # Get AAAA records
    try:
        aaaa_records = dns.resolver.resolve(domain, 'AAAA')
        records['AAAA'] = [aaaa.address for aaaa in aaaa_records]
    except Exception as e:
        records['AAAA'] = f"Error: {e}"
    
    # Get NS records
    try:
        ns_records = dns.resolver.resolve(domain, 'NS')
        records['NS'] = [ns.to_text() for ns in ns_records]
    except Exception as e:
        records['NS'] = f"Error: {e}"
    
    # Get TXT records
    try:
        txt_records = dns.resolver.resolve(domain, 'TXT')
        records['TXT'] = [txt.to_text() for txt in txt_records]
    except Exception as e:
        records['TXT'] = f"Error: {e}"
    
    return records

def verify_email(email):
    if not is_valid_email_format(email):
        print("Invalid email format")
        return False, "Invalid email format"
    
    domain = email.split('@')[1]
    dns_records = get_dns_records(domain)
    if not dns_records:
        print("No DNS records found for domain")
        return False, "No DNS records found for domain"
    
    #for record_type, values in dns_records.items():
    #    print(f"{record_type} records: {values}")
    
    # Try connecting to each MX server
    for mx in dns_records['MX']:
        try:
            server = smtplib.SMTP(mx)
            server.set_debuglevel(0)
            server.helo()
            server.mail('test@example.com')
            code, message = server.rcpt(email)
            server.quit()
            if code == 250:
                dns_records['EMAIL_EXISTS'] = True
                return True, dns_records
        except Exception as e:
            print(f"Error verifying email: {e}")
    dns_records['EMAIL_EXISTS'] = False
    return True, dns_records

In [ ]:
# Example usage
email = "benvanceuf@gmail.com"
dns_high_level, dns_details = verify_email(email)

In [ ]:
dns_high_level

In [ ]:
dns_details